In [21]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("data/")
    return document_loader.load()

In [22]:
documents = load_documents()

In [23]:
print(documents[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents : list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100,
        length_function = len,
        is_separator_regex = False,
    )
    
    return text_splitter.split_documents(documents)


In [25]:
chunks = split_documents(documents)
print(chunks[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [7]:

from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\SMC-HP\anaconda3\envs\cb\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [17]:
#run this if you want to delte the already created chroma database
'''
import os
import shutil
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)
'''


In [5]:
from langchain.vectorstores.chroma import Chroma
CHROMA_PATH = "chroma"

In [27]:
#run this if chroma database is not already created

db = Chroma.from_documents(
    chunks,embeddings_model,persist_directory=CHROMA_PATH
)

In [8]:
#connecting to existing database
db = Chroma(persist_directory=CHROMA_PATH,embedding_function=embeddings_model)

Human: 
Answer the questio based only on the following context:
• Heredity. Individuals with a family history of acne have
greater susceptibility to the disease.
• Hormonal changes. Acne can flare up before menstrua-
tion, during pregnancy , and menopause .
• Diet. No foods cause acne, but certain foods may cause
flare-ups.
• Drugs. Acne can be a side effect of drugs including
tranquilizers, antidepressants, antibiotics ,oral contra-
ceptives , and anabolic steroids.
• Personal hygiene. Abrasive soaps, hard scrubbing, or
picking at pimples will make them worse.
• Cosmetics. Oil-based makeup and hair sprays worsen
acne.
• Environment. Exposure to oils and greases, polluted air,
and sweating in hot weather aggravate acne.
•Stress . Emotional stress may contribute to acne.
Acne is usually not conspicuous, although inflamed
lesions may cause pain , tenderness, itching , or swelling.

---

and bursts open, releasing sebum, bacteria, and skin andwhite blood cells into the surrounding tissues

Unfortunately, I cannot diagnose or treat any medical conditions, including migraines. However, I can provide some general information and advice on how to manage a headache.

If you are experiencing a headache, there are several things you can try to alleviate the pain:

1. Rest and relaxation: Sometimes, the best thing to do is to rest and relax. Lie down in a quiet, dark room and try to relax. This can help reduce stress and tension, which can contribute to headaches.
2. Cold or warm compresses: Applying a cold or warm compress to the forehead or neck may help relieve tension and reduce pain.
3. Over-the-counter pain medication: Nonsteroidal anti-inflammatory drugs (NSAIDs) such as ibuprofen or acetaminophen can help reduce pain and inflammation. Always follow the recommended dosage instructions and consult with a doctor before taking any medication.
4. Hydration: Dehydration can exacerbate headaches, so drinking plenty of water throughout the day can help prevent and treat headache

In [15]:
import pickle

In [20]:
pickle.dump(embeddings_model, open('model.pkl', 'wb'))